In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
# project_dir = "/content/drive/MyDrive/AraGenEval2025"
# os.chdir(project_dir)
# print(f"✅ Current directory: {os.getcwd()}")


In [3]:
!pip install transformers[sentencepiece] torch pandas openpyxl psutil python-dotenv peft  --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 6.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 39.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 22.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 11.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 34.5 MB/s eta 0:00:0000:0100:01


In [4]:
import os

path = "/kaggle/working"
is_empty = (len(os.listdir(path)) < 5)

!ls /kaggle/working/

MODEL_WEIGHTS_SRC ="/kaggle/input/ubc-nlp-rat5v2-base-1024-model/UBC-NLP_AraT5v2-base-1024/final"+"/*"

lora_injection_src=  "/kaggle/input/ubc-ara-t5-v2-lora-ast-model/UBC-NLP_AraT5v2-base-1024_with_lora/UBC-NLP_AraT5v2-base-1024_with_lora/final"+"/*"

# MODEL_NAME = "facebook_mbart-large-50-many-to-many-mmt" #"agemagician_mlong-t5-tglobal-large" #"facebook_mbart-large-50-many-to-many-mmt" # "UBC-NLP_AraT5-base" #"google/mt5-small"
MODEL_NAME = "UBC-NLP_AraT5v2-base-1024" 

MODEL_WEIGHTS_DEST_DIR =f"/kaggle/working/models/fine_tuned_model/model_weights/{MODEL_NAME}"

base_model_path = MODEL_WEIGHTS_DEST_DIR
lora_path       = f"./{MODEL_NAME}_with_lora"

In [5]:
## copy project structure and other needed inputs

if is_empty:
    print(f"✅ '{path}' is empty. copying project strcuture into it ...")
    
    #  project strcture
    !cp -r /kaggle/input/arageneval2025-task1-sc/* /kaggle/working/
    
    # final test dataset
    !cp /kaggle/input/arageneval2025-task1-finaldataset/PublicDataFinalPhaseTask1.xlsx /kaggle/working/data/
    
    # model weights
    !mkdir -p {MODEL_WEIGHTS_DEST_DIR}
    !cp -r {MODEL_WEIGHTS_SRC} {MODEL_WEIGHTS_DEST_DIR}
    #lora weights
    # !mkdir -p {lora_path}
    # !cp -r {lora_injection_src} {lora_path}
else:
    print(f"❌ '{path}' is not empty — contains {len(os.listdir(path))} items.")

✅ '/kaggle/working' is empty. copying project strcuture into it ...


In [6]:
# Change to project directory
%cd /kaggle/working/ 

# Confirm
!ls

/kaggle/working
config.py  evaluation	models	 run_pipeline.py
data	   __init__.py	prompts  utils


In [7]:
!ls -R {MODEL_WEIGHTS_DEST_DIR}

/kaggle/working/models/fine_tuned_model/model_weights/UBC-NLP_AraT5v2-base-1024:
added_tokens.json	model.safetensors	 tokenizer_config.json
config.json		special_tokens_map.json  training_args.bin
generation_config.json	spiece.model


In [8]:
!ls -R {lora_path}

ls: cannot access './UBC-NLP_AraT5v2-base-1024_with_lora': No such file or directory


In [9]:
# Ensure PyTorch can expand allocations if needed
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [10]:

import os
import json
import dotenv
import torch
import pandas as pd
from datetime import datetime
from tqdm import tqdm
import gc
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

from config import Config


# project root in PYTHONPATH
import sys
project_root = os.getcwd()
if project_root not in sys.path:
    sys.path.insert(0, project_root)


# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# print(f"✅ Using device: {device}")

# Device & parallel setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"✅ Using device: {device}, count: {torch.cuda.device_count()} GPUs")

✅ Using device: cuda, count: 1 GPUs


In [11]:

def safe_model_name(model_name):
    return model_name.replace("/", "_")

# archive & inspect
def create_run_dir(base, name):
    ts = datetime.now().strftime("%Y%m%d_%H%M%S")
    d  = os.path.join(base, safe_model_name(name), f"run_{ts}")
    os.makedirs(d, exist_ok=True)
    return d

def run_evaluation():
  pass
    # os.system("python utils/prepare_metrics_log.py")
    # os.system("python evaluation/evaluate.py")


def archive_results(run_dir, files_to_archive):
    for fpath in files_to_archive:
        if os.path.exists(fpath):
            dest = os.path.join(run_dir, os.path.basename(fpath))
            os.system(f"cp {fpath} {dest}")


In [12]:
def load_model(model_name: str):
    secure = safe_model_name(model_name)
    model_path = os.path.join("models/fine_tuned_model/model_weights", secure)
    print(f"Loading model from {model_path}")
    tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=False, legacy=False)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_path)
    # mBART language tags
    if "bart" in model_name.lower():
        tokenizer.src_lang = tokenizer.tgt_lang = "ar_AR"
    # wrap for multi‑GPU
    if torch.cuda.device_count() > 1:
        model = torch.nn.DataParallel(model)
    model.to(device).eval()
    return tokenizer, model  

In [13]:
from peft import PeftModel
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

def load_lora_model(base_model_path: str, lora_path: str):
    print(f"Loading base model from {base_model_path}")
    print(f"Applying LoRA adapters from {lora_path}")

    # Tokenizer
    tokenizer = AutoTokenizer.from_pretrained(base_model_path, use_fast=False, legacy=False)

    # Base model
    base_model = AutoModelForSeq2SeqLM.from_pretrained(base_model_path)

    # Apply LoRA
    model = PeftModel.from_pretrained(base_model, lora_path)

    # Optional: for mBART-like models
    if "bart" in base_model_path.lower():
        tokenizer.src_lang = tokenizer.tgt_lang = "ar_AR"

    # DataParallel
    if torch.cuda.device_count() > 1:
        model = torch.nn.DataParallel(model)

    model.to(device).eval()
    return tokenizer, model

2025-07-24 14:55:49.996639: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753368950.167907      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753368950.218593      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [14]:
def save_predictions_to_file(records, df, output_log_path):
    # Map records by ID
    records_by_id = {r["id"]: r for r in records}

    # Re-order by original ID order
    ordered = [records_by_id[i] for i in df["id"] if i in records_by_id]
    missing = [i for i in df["id"] if i not in records_by_id]
    if missing:
        print(f"⚠️ Missing {len(missing)} records: {missing[:10]}...")

    # Save
    with open(output_log_path, "w", encoding="utf-8") as f:
        json.dump(ordered, f, ensure_ascii=False, indent=2)
    print(f"✅ Generated {len(ordered)} records saved to {output_log_path}")


In [15]:
# ### Batched inference function (robust, with error recovery)
# def generate_predictions(tokenizer, model, validation_path, output_log_path, batch_size=8):
#     import time
#     import traceback

#     # 1. Load & prepare
#     df = pd.read_excel(validation_path)
#     df["input_text"] = df["author"].astype(str) + " [SEP] " + df["text_in_msa"].astype(str)

#     # Handle DataParallel wrapping
#     base_model = model.module if hasattr(model, "module") else model
#     base_model.generation_config.pad_token_id = tokenizer.pad_token_id
#     assert base_model.config.eos_token_id is not None

#     records = []

#     from torch.utils.data import DataLoader, Dataset

#     class TextDataset(Dataset):
#         def __init__(self, texts): self.texts = texts
#         def __len__(self): return len(self.texts)
#         def __getitem__(self, idx): return self.texts[idx]

#     def collate_fn(batch_texts):
#         return tokenizer(
#             batch_texts,
#             return_tensors="pt",
#             padding=True,
#             truncation=True,
#             max_length=1024
#         )

#     loader = DataLoader(
#         TextDataset(df["input_text"].tolist()),
#         batch_size=batch_size,
#         shuffle=False,
#         collate_fn=collate_fn,
#         num_workers=4,
#         pin_memory=True
#     )

#     for i, batch in enumerate(tqdm(loader, desc="Batches")):
#         success = False
#         attempts = 0
#         while not success and attempts < 3:
#             try:
#                 batch = {k: v.to(device) for k, v in batch.items()}
#                 with torch.inference_mode():
#                     gen_ids = base_model.generate(
#                         input_ids=batch["input_ids"],
#                         attention_mask=batch["attention_mask"],
#                         max_new_tokens=1024
#                     )
#                 outs = tokenizer.batch_decode(gen_ids, skip_special_tokens=True)

#                 start_idx = i * batch_size
#                 for j, out in enumerate(outs):
#                     row = df.iloc[start_idx + j]
#                     records.append({
#                         "id": int(row["id"]),
#                         "author": row["author"],
#                         "output": out,
#                         "ground_truth": row["text_in_author_style"]
#                     })
#                 success = True

#             except RuntimeError as e:
#                 attempts += 1
#                 print(f"❌ RuntimeError on batch {i}, attempt {attempts}: {str(e)}")
#                 traceback.print_exc()

#                 torch.cuda.empty_cache()
#                 gc.collect()

#                 if "CUDA out of memory" in str(e):
#                     print("⚠️ Reducing batch size temporarily and retrying...")
#                     time.sleep(2)
#                     if batch["input_ids"].shape[0] > 1:
#                         # Retry each sample individually
#                         for j in range(batch["input_ids"].shape[0]):
#                             try:
#                                 single_batch = {k: v[j:j+1] for k, v in batch.items()}
#                                 with torch.inference_mode():
#                                     gen_ids = base_model.generate(
#                                         input_ids=single_batch["input_ids"],
#                                         attention_mask=single_batch["attention_mask"],
#                                         max_new_tokens=1024
#                                     )
#                                 out = tokenizer.decode(gen_ids[0], skip_special_tokens=True)
#                                 row = df.iloc[i * batch_size + j]
#                                 records.append({
#                                     "id": int(row["id"]),
#                                     "author": row["author"],
#                                     "output": out,
#                                     "ground_truth": row["text_in_author_style"]
#                                 })
#                             except Exception as inner_e:
#                                 print(f"⚠️ Skipping failed sample {i * batch_size + j} id = {int(row['id'])}: {str(inner_e)}")
#                                 traceback.print_exc()
#                     success = True  # Continue to next batch regardless
#                 else:
#                     raise e

#         torch.cuda.empty_cache()
#         gc.collect()

#     # 4. Save
#     with open(output_log_path, "w", encoding="utf-8") as f:
#         json.dump(records, f, ensure_ascii=False, indent=2)
#     print(f"✅ Generated {len(records)} records to {output_log_path}")


In [16]:
# def generate_predictions(tokenizer, model, validation_path, output_log_path, batch_size=8):
#     import time, gc, json, traceback
#     import torch
#     from torch.utils.data import DataLoader, Dataset
#     from tqdm import tqdm
#     import pandas as pd

#     max_input_length=1300
#     max_generation_length = 1500

#     # 1. Load & prepare
#     df = pd.read_excel(validation_path)
#     df["input_text"] = df["author"].astype(str) + " [SEP] " + df["text_in_msa"].astype(str)
    
#     # Compute token lengths
#     df["token_len"] = df["input_text"].apply(lambda x: len(tokenizer.encode(x, truncation=True, max_length=1024)))
    
#     # Save original order
#     original_order = df[["id"]].copy()

#     # Sort by token length
#     df_sorted = df.sort_values("token_len").reset_index(drop=True)

#     # Handle DataParallel wrapping
#     base_model = model.module if hasattr(model, "module") else model
#     base_model.generation_config.pad_token_id = tokenizer.pad_token_id
#     assert base_model.config.eos_token_id is not None

#     records = []

#     class TextDataset(Dataset):
#         def __init__(self, texts): self.texts = texts
#         def __len__(self): return len(self.texts)
#         def __getitem__(self, idx): return self.texts[idx]

#     def collate_fn(batch_texts):
#         return tokenizer(
#             batch_texts,
#             return_tensors="pt",
#             padding=True,
#             truncation=True,
#             max_length=max_input_length
#         )

#     loader = DataLoader(
#         TextDataset(df_sorted["input_text"].tolist()),
#         batch_size=batch_size,
#         shuffle=False,
#         collate_fn=collate_fn,
#         num_workers=2,
#         pin_memory=True
#     )

#     for i, batch in enumerate(tqdm(loader, desc="Batches")):
#         success = False
#         attempts = 0
#         while not success and attempts < 3:
#             try:
#                 batch = {k: v.to(device) for k, v in batch.items()}
#                 with torch.inference_mode():
#                     gen_ids = base_model.generate(
#                         input_ids=batch["input_ids"],
#                         attention_mask=batch["attention_mask"],
#                         max_new_tokens=max_generation_length,
#                         use_cache=False
#                     )
#                 outs = tokenizer.batch_decode(gen_ids, skip_special_tokens=True)

#                 start_idx = i * batch_size
#                 for j, out in enumerate(outs):
#                     row = df_sorted.iloc[start_idx + j]
#                     records.append({
#                         "id": int(row["id"]),
#                         "author": row["author"],
#                         "output": out,
#                         # "ground_truth": row["text_in_author_style"]
#                     })
                
#                 # Proactively clear memory between batches
#                 del gen_ids, outs
#                 torch.cuda.empty_cache()
#                 gc.collect()
                
#                 success = True

#             except RuntimeError as e:
#                 attempts += 1
#                 print(f"❌ RuntimeError on batch {i}, attempt {attempts}: {str(e)}")
#                 traceback.print_exc()
#                 torch.cuda.empty_cache()
#                 gc.collect()

#                 if "CUDA out of memory" in str(e):
#                     print("⚠️ Reducing batch size temporarily and retrying...")
#                     time.sleep(2)
#                     if batch["input_ids"].shape[0] > 1:
#                         for j in range(batch["input_ids"].shape[0]):
#                             try:
#                                 single_batch = {k: v[j:j+1] for k, v in batch.items()}
#                                 with torch.inference_mode():
#                                     gen_ids = base_model.generate(
#                                         input_ids=single_batch["input_ids"],
#                                         attention_mask=single_batch["attention_mask"],
#                                         max_new_tokens=max_generation_length,
#                                         use_cache=False
#                                     )
#                                 out = tokenizer.decode(gen_ids[0], skip_special_tokens=True)
#                                 row = df_sorted.iloc[i * batch_size + j]
#                                 records.append({
#                                     "id": int(row["id"]),
#                                     "author": row["author"],
#                                     "output": out,
#                                     # "ground_truth": row["text_in_author_style"]
#                                 })
#                             except Exception as inner_e:
#                                 print(f"⚠️ Skipping failed sample {i * batch_size + j} id = {int(row['id'])}: {str(inner_e)}")
#                                 traceback.print_exc()
#                     success = True
#                 else:
#                     raise e

#         torch.cuda.empty_cache()
#         gc.collect()

#     # Restore original order
#     df_records = pd.DataFrame(records)
#     df_final = original_order.merge(df_records, on="id", how="left")

#     with open(output_log_path, "w", encoding="utf-8") as f:
#         json.dump(df_final.to_dict(orient="records"), f, ensure_ascii=False, indent=2)
#     print(f"✅ Generated {len(df_final)} records to {output_log_path}")


In [17]:
def generate_predictions(tokenizer, model, validation_path, output_log_path, batch_size=8):
    import time, gc, json, traceback
    import torch
    from tqdm import tqdm
    import pandas as pd

    max_input_length = 3350
    max_generation_length = 3500

    # Load and prepare
    df = pd.read_excel(validation_path)
    df["input_text"] = df["author"].astype(str) + " [SEP] " + df["text_in_msa"].astype(str)
    df["token_len"] = df["input_text"].apply(lambda x: len(tokenizer.encode(x, truncation=True, max_length=1024)))
    original_order = df[["id"]].copy()
    df_sorted = df.sort_values("token_len").reset_index(drop=True)

    # Handle DataParallel
    base_model = model.module if hasattr(model, "module") else model
    base_model.generation_config.pad_token_id = tokenizer.pad_token_id
    assert base_model.config.eos_token_id is not None

    device = next(base_model.parameters()).device
    records = []

    def collate_fn(batch_texts):
        return tokenizer(
            batch_texts,
            return_tensors="pt",
            padding="longest",
            truncation=True,
            max_length=max_input_length
        )

    current_batch_size = batch_size  # Will be updated globally

    def process_batch(batch_texts, start_idx):
        nonlocal records, current_batch_size
        try:
            tokenized = collate_fn(batch_texts)
            input_lens = tokenized["attention_mask"].sum(dim=1)
            dynamic_max_gen_len = min(max_generation_length, input_lens.max().item() + 256)
            tokenized = {k: v.to(device) for k, v in tokenized.items()}

            with torch.inference_mode():
                gen_ids = base_model.generate(
                    input_ids=tokenized["input_ids"],
                    attention_mask=tokenized["attention_mask"],
                    max_new_tokens=dynamic_max_gen_len,
                    use_cache=False
                )
            outputs = tokenizer.batch_decode(gen_ids, skip_special_tokens=True)

            for j, out in enumerate(outputs):
                row = df_sorted.iloc[start_idx + j]
                records.append({
                    "id": int(row["id"]),
                    "author": row["author"],
                    "output": out,
                })

            del gen_ids, outputs, tokenized
            torch.cuda.empty_cache()
            gc.collect()
            return True

        except RuntimeError as e:
            if "CUDA out of memory" in str(e) and len(batch_texts) > 1:
                torch.cuda.empty_cache()
                gc.collect()
                mid = len(batch_texts) // 2
                print(f"⚠️ OOM on batch [{start_idx}:{start_idx+len(batch_texts)}]. Retrying halves of size {mid}...")

                left_success = process_batch(batch_texts[:mid], start_idx)
                right_success = process_batch(batch_texts[mid:], start_idx + mid)

                # Update global batch size to the successful smaller value
                if left_success and right_success and mid < current_batch_size:
                    print(f"✅ Updating future batch size to {mid}")
                    current_batch_size = mid

                return left_success and right_success
            else:
                print(f"❌ Failed batch [{start_idx}:{start_idx+len(batch_texts)}]: {str(e)}")
                traceback.print_exc()
                return False

    # Manual batching loop with adaptive batch size
    i = 0
    total = len(df_sorted)
    input_texts = df_sorted["input_text"].tolist()

    with tqdm(total=total, desc="Batches") as pbar:
        while i < total:
            end = min(i + current_batch_size, total)
            batch_texts = input_texts[i:end]
            success = process_batch(batch_texts, i)
            if success:
                pbar.update(len(batch_texts))
                i += len(batch_texts)
            else:
                print(f"⚠️ Skipping batch [{i}:{end}] completely after failure.")
                i += len(batch_texts)  # skip failed samples

    # Restore original order
    df_records = pd.DataFrame(records)
    df_final = original_order.merge(df_records, on="id", how="left")

    with open(output_log_path, "w", encoding="utf-8") as f:
        json.dump(df_final.to_dict(orient="records"), f, ensure_ascii=False, indent=2)

    print(f"✅ Generated {len(df_final)} records to {output_log_path}")


In [18]:
# # Single‑GPU token‑budgeted generation with last‑three‑batches shrink
# import math
# import threading
# from queue import Queue

# def generate_predictions_by_budget(
#     tokenizer,
#     model,
#     validation_path,
#     output_log_path,
#     token_budget=30_000,
#     budget_update_freq=5,
#     budget_update_quota=2000,
#     memory_usage_threshold=80
# ):
#     import time, gc, json, traceback
#     import torch
#     import pandas as pd
#     from tqdm import tqdm
#     import pynvml
#     from transformers import DataCollatorWithPadding

#     # --- Init NVML & Model ---
#     pynvml.nvmlInit()
#     device = next(model.module.parameters() if hasattr(model, "module") else model.parameters()).device
#     handle = pynvml.nvmlDeviceGetHandleByIndex(device.index or 0)
#     total_vram_mb = pynvml.nvmlDeviceGetMemoryInfo(handle).total / 1e6

#     max_input_length, max_generation_length = 3350, 3500

#     # --- Load & Pre‑tokenize ---
#     df = pd.read_excel(validation_path)
#     df["input_text"] = df["author"].astype(str) + " [SEP] " + df["text_in_msa"].astype(str)
#     texts = df["input_text"].tolist()
#     encodings = tokenizer(texts, truncation=True, max_length=max_input_length,
#                           return_attention_mask=True, padding=False)
#     token_lens = [len(ids) for ids in encodings["input_ids"]]
#     original = df[["id"]].copy()
#     order = sorted(range(len(texts)), key=lambda i: token_lens[i])

#     collator = DataCollatorWithPadding(tokenizer, padding="longest")
#     base_model = model.module if hasattr(model, "module") else model
#     base_model.generation_config.pad_token_id = tokenizer.pad_token_id
#     base_model.to(device)

#     # --- 1) Pre‑compute batches under token_budget ---
#     raw_batches = []
#     i = 0
#     while i < len(order):
#         batch_idxs, sum_toks = [], 0
#         while i < len(order) and sum_toks + token_lens[order[i]] <= token_budget:
#             batch_idxs.append(order[i])
#             sum_toks += token_lens[order[i]]
#             i += 1
#         if not batch_idxs:
#             batch_idxs = [order[i]]
#             sum_toks = token_lens[order[i]]
#             i += 1
#         raw_batches.append((batch_idxs, sum_toks))

#     # --- 2) Shrink the last three batches to 20% size ---
#     def chunk_list(lst, pct):
#         size = max(1, math.ceil(len(lst) * pct))
#         return [lst[j:j+size] for j in range(0, len(lst), size)]

#     for k in (-1):
#         if 0 <= k + len(raw_batches) < len(raw_batches):
#             idxs, _ = raw_batches[k]
#             shrunk = chunk_list(idxs, 0.2)
#             raw_batches[k] = [(chunk, sum(token_lens[j] for j in chunk)) for chunk in shrunk]

#     # Flatten into final batch list
#     final_batches = []
#     for entry in raw_batches:
#         if isinstance(entry[0], list):  # single batch
#             final_batches.append(entry)
#         else:
#             # in case we replaced last three with multiple chunks
#             for chunk_idxs, sum_toks in entry:
#                 final_batches.append((chunk_idxs, sum_toks))

#     # --- 3) Run generation with CPU/GPU overlap on final_batches ---
#     records = []
#     current_budget = token_budget
#     clean_batches = 0

#     batch_queue = Queue(maxsize=2)
#     stop_event = threading.Event()

#     def batch_producer():
#         for batch_idxs, sum_toks in final_batches:
#             batch_encoding = {k: [encodings[k][j] for j in batch_idxs] for k in encodings}
#             batch = collator(batch_encoding)
#             batch_queue.put((batch, batch_idxs, sum_toks))
#             if stop_event.is_set():
#                 break
#         batch_queue.put(None)

#     threading.Thread(target=batch_producer, daemon=True).start()

#     try:
#         pbar = tqdm(total=len(final_batches))
#         for item in iter(batch_queue.get, None):
#             batch, batch_idxs, tok_sum = item
#             batch = {k: v.to(device) for k, v in batch.items()}
#             lengths = batch["attention_mask"].sum(1)
#             gen_max = min(max_generation_length, lengths.max().item() + 256)

#             try:
#                 torch.cuda.synchronize()
#                 t0 = time.time()
#                 out_ids = base_model.generate(
#                     input_ids=batch["input_ids"],
#                     attention_mask=batch["attention_mask"],
#                     max_new_tokens=gen_max,
#                     use_cache=True,
#                     num_beams=1
#                 )
#                 torch.cuda.synchronize()
#                 dt = time.time() - t0

#                 mem = pynvml.nvmlDeviceGetMemoryInfo(handle).used / 1e6
#                 pct = mem / total_vram_mb * 100
#                 print(f"[Gen] {len(batch_idxs)} samples | Time: {dt:.1f}s | VRAM: {mem:.0f}MB ({pct:.0f}%)")

#                 outs = tokenizer.batch_decode(out_ids.cpu(), skip_special_tokens=True)
#                 for idx, text in zip(batch_idxs, outs):
#                     records.append({
#                         "id": int(df.at[idx, "id"]),
#                         "author": df.at[idx, "author"],
#                         "output": text
#                     })
#                 clean_batches += 1
#                 pbar.update(1)

#             except RuntimeError as e:
#                 if "out of memory" in str(e).lower() and len(batch_idxs) > 1:
#                     torch.cuda.empty_cache(); gc.collect()
#                     current_budget = max(current_budget // 2, tok_sum)
#                     clean_batches = 0
#                     print(f"⚠️ OOM -> new budget {current_budget}")
#                     # split & requeue
#                     mid = len(batch_idxs) // 2
#                     for half in (batch_idxs[:mid], batch_idxs[mid:]):
#                         half_sum = sum(token_lens[j] for j in half)
#                         batch_queue.put((collator({k: [encodings[k][j] for j in half] for k in encodings}), half, half_sum))
#                 else:
#                     print(f"Error: {e}"); traceback.print_exc()
#                     clean_batches = 0

#             # budget bump
#             if clean_batches >= budget_update_freq and pct < memory_usage_threshold:
#                 current_budget += budget_update_quota
#                 print(f"⚡ budget -> {current_budget}")
#                 clean_batches = 0

#         pbar.close()

#     except KeyboardInterrupt:
#         print("⚠️ Interrupted — saving partial results...")

#     finally:
#         stop_event.set()
#         # Save results
#         try:
#             df_out = original.merge(pd.DataFrame(records), on="id", how="left")
#             with open(output_log_path, "w", encoding="utf-8") as f:
#                 json.dump(df_out.to_dict(orient="records"), f, ensure_ascii=False, indent=2)
#             print(f"✅ Saved {len(records)} records")
#         except Exception as ex:
#             print(f"❌ Save failed: {ex}")
#         finally:
#             pynvml.nvmlShutdown()

#     return records


In [19]:
# !pip install accelerate
# accelerate config      # answer prompts (choose multi‑GPU)

In [20]:
# import os
# from config import Config  # import your custom Config module
# import pandas as pd
# import torch
# gc = __import__('gc')
# from tqdm import tqdm
# from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
# from accelerate import Accelerator
# import pandas as pd
# import torch
# gc = __import__('gc')
# from tqdm import tqdm
# from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
# from accelerate import Accelerator

# def generate_predictions(
#     tokenizer, model, validation_path, output_log_path,
#     max_new_tokens=1700,
#     token_budget=100_000,
#     sort_inputs=True
# ):
#     # Initialize Accelerator
#     accelerator = Accelerator()
#     # Prepare model & tokenizer
#     model, tokenizer = accelerator.prepare(model, tokenizer)
#     model.eval()
#     unwrapped = accelerator.unwrap_model(model)

#     # 1. Load & prepare DataFrame
#     df = pd.read_excel(validation_path)
#     df['input_text'] = df['author'].astype(str) + ' [SEP] ' + df['text_in_msa'].astype(str)

#     # 2. Compute token lengths
#     max_len = 1024 #unwrapped.config.max_position_embeddings
#     enc = tokenizer(
#         df['input_text'].tolist(), return_length=True,
#         padding=False, truncation=True, max_length=max_len
#     )
#     lengths = enc['length']

#     # 3. Build token-budgeted batches
#     indices = list(range(len(lengths)))
#     if sort_inputs:
#         indices.sort(key=lambda i: lengths[i], reverse=True)
#     batches, curr, curr_sum = [], [], 0
#     for i in indices:
#         cost = lengths[i] + max_new_tokens
#         if curr and curr_sum + cost > token_budget:
#             batches.append(curr)
#             curr, curr_sum = [], 0
#         curr.append(i)
#         curr_sum += cost
#     if curr:
#         batches.append(curr)

#     # 4. Generation loop with instrumentation
#     records = []
#     for batch_idx, batch_ids in enumerate(tqdm(batches, desc='Accelerated Batches')):
#         # Instrumentation
#         sample_count = len(batch_ids)
#         batch_token_cost = sum(lengths[i] + max_new_tokens for i in batch_ids)
#         print(f"\n▶ Batch {batch_idx+1}/{len(batches)}: samples={sample_count}, token_cost={batch_token_cost}")
#         # Memory before
#         for dev in range(torch.cuda.device_count()):
#             alloc = torch.cuda.memory_allocated(dev) / 1024**2
#             reserved = torch.cuda.memory_reserved(dev) / 1024**2
#             print(f"   • GPU {dev}: allocated={alloc:.1f} MiB, reserved={reserved:.1f} MiB")

#         texts = [df.iloc[i]['input_text'] for i in batch_ids]
#         inputs = tokenizer(
#             texts, return_tensors='pt', padding=True,
#             truncation=True, max_length=max_len
#         )
#         inputs = {k: v.to(accelerator.device) for k, v in inputs.items()}

#         with torch.no_grad():
#             gen_ids = model.generate(
#                 input_ids=inputs['input_ids'],
#                 attention_mask=inputs['attention_mask'],
#                 max_new_tokens=max_new_tokens
#             )
#         gen_ids = accelerator.gather(gen_ids)
#         outs = tokenizer.batch_decode(gen_ids, skip_special_tokens=True)

#         # Memory after
#         for dev in range(torch.cuda.device_count()):
#             alloc = torch.cuda.memory_allocated(dev) / 1024**2
#             reserved = torch.cuda.memory_reserved(dev) / 1024**2
#             print(f"   • GPU {dev}: allocated={alloc:.1f} MiB, reserved={reserved:.1f} MiB")

#         for idx, out in zip(batch_ids, outs):
#             row = df.iloc[idx]
#             records.append({
#                 'id': int(row['id']),
#                 'author': row['author'],
#                 'output': out,
#                 'ground_truth': row['text_in_author_style']
#             })

#         # cleanup
#         del inputs, gen_ids
#         gc.collect()
#         torch.cuda.empty_cache()

#     # 5. Save results
#     import json
#     with open(output_log_path, 'w', encoding='utf-8') as f:
#         json.dump(records, f, ensure_ascii=False, indent=2)
#     print(f"✅ Generated {len(records)} records and saved to {output_log_path}")
#     return records, df



In [21]:
# Single-GPU token-budgeted generation (pre-tokenized + efficient CPU/GPU overlap with OOM retry)
import threading
from queue import Queue

def generate_predictions_by_budget(
    tokenizer,
    model,
    validation_path,
    output_log_path,
    token_budget=10_000,
    budget_update_freq=5,
    budget_update_quota=1000,
    memory_usage_threshold=80
):
    import time, gc, json, traceback
    import torch
    import pandas as pd
    from tqdm import tqdm
    import pynvml
    from transformers import DataCollatorWithPadding

    pynvml.nvmlInit()
    device = next(model.module.parameters() if hasattr(model, "module") else model.parameters()).device
    handle = pynvml.nvmlDeviceGetHandleByIndex(device.index or 0)
    total_vram_mb = pynvml.nvmlDeviceGetMemoryInfo(handle).total / 1e6

    max_input_length = 3400
    max_generation_length = 4000
    max_pos = 100000000 #getattr(tokenizer, 'model_max_length', None) or model.config.max_position_embeddings
    safe_input_length = min(max_input_length, max_pos)
    safe_gen_length = min(max_generation_length, max_pos)
    print(f"▶ model.max_position_embeddings = {max_pos} , safe_input = {safe_input_length}, safe_generation = {safe_gen_length}")

    print("loading and tokenization start ...")
    df = pd.read_excel(validation_path)
    df["input_text"] = df["author"].astype(str) + " [SEP] " + df["text_in_msa"].astype(str)
    texts = df["input_text"].tolist()
    encodings = tokenizer(
        texts,
        truncation=True,
        max_length=safe_input_length,
        return_attention_mask=True,
        padding=False,
    )
    token_lens = [len(ids) for ids in encodings["input_ids"]]
    df["token_len"] = token_lens
    original = df[["id"]].copy()
    order = sorted(range(len(texts)), key=lambda i: token_lens[i])
    collator = DataCollatorWithPadding(tokenizer, padding="longest")

    print("loading and tokenization done ... loading the model to gpu...")
    base_model = model.module if hasattr(model, "module") else model
    base_model.generation_config.pad_token_id = tokenizer.pad_token_id
    base_model.to(device)

    records = []
    current_budget = token_budget
    clean_batches = 0

    batch_queue = Queue(maxsize=2)
    stop_event = threading.Event()

    def batch_producer():
        i = 0
        while not stop_event.is_set() and i < len(order):
            batch_idxs, sum_toks = [], 0
            while i < len(order) and sum_toks + token_lens[order[i]] <= current_budget:
                batch_idxs.append(order[i])
                sum_toks += token_lens[order[i]]
                i += 1

            # Catch edge case: one sample too long
            if not batch_idxs and i < len(order):
                batch_idxs = [order[i]]
                sum_toks = token_lens[order[i]]
                i += 1

            if batch_idxs:
                batch_encoding = {k: [encodings[k][j] for j in batch_idxs] for k in encodings}
                batch = collator(batch_encoding)
                batch_queue.put((batch, batch_idxs, sum_toks, i))
            else:
                break  # nothing left to process

        batch_queue.put(None)  # signal end

    prod_thread = threading.Thread(target=batch_producer)
    prod_thread.start()

    try:
        pbar = tqdm(total=len(order))
        while True :
            n=0
            print(f"retrieve batch {n} ...")
            item = batch_queue.get()
            if item is None:
                if len(records) >= len(order):
                    break
                else:
                    time.sleep(0.1)
                    continue

            print(f"start batch {n} processing...")
            
            batch, batch_idxs, tok_sum, _ = item
            batch = {k: v.to(device) for k, v in batch.items()}  # FIXED
            lengths = batch["attention_mask"].sum(1)

            # gen_max = min(safe_gen_length, lengths.max().item() + 256)

            # Precomputed from your EDA:
            p25_ratio = 0.9291   # 25% quantile of (target/input)
            p75_ratio = 1.0244   # 75% quantile
            
            # Add a small buffer around those:
            min_frac = max(0.0, p25_ratio - 0.05)   # ~0.88
            max_frac = p75_ratio + 0.10             # ~1.12
            
            # In your batch loop:
            # batch_input_lens = batch["attention_mask"].sum(dim=1)   # tensor of per-sample
            max_in = int(lengths.max().item())
            
            min_new = int(max_in * min_frac)
            max_new = int(max_in * max_frac)
            
            
            gen_max = min(safe_gen_length, max_new)

            print(f" min gen = {min_new} , max_new={max_new} , max_gen = {gen_max}")

            try:
                torch.cuda.synchronize()
                t0 = time.time()
                out_ids = base_model.generate(
                    input_ids=batch["input_ids"],
                    attention_mask=batch["attention_mask"],              
                    # always use cache for speed
                    use_cache = True,
                    num_beams=3,
                    # style‑aware length windows
                    min_new_tokens = min_new,
                    max_new_tokens=gen_max,
                    # discourage repeats
                    no_repeat_ngram_size = 3,
                    repetition_penalty    = 1.1,
                    # length bias
                    length_penalty = 0.9,
                    # batching can still parallelize
                    pad_token_id = tokenizer.pad_token_id,
                    eos_token_id = tokenizer.eos_token_id,
                )
                torch.cuda.synchronize()
                dt = time.time() - t0

                mem = pynvml.nvmlDeviceGetMemoryInfo(handle).used / 1e6
                pct = mem / total_vram_mb * 100
                print(f"[Gen] {len(batch_idxs)} samples | Time: {dt:.1f}s | VRAM: {mem:.0f}MB ({pct:.0f}%)")

                outs = tokenizer.batch_decode(out_ids.cpu(), skip_special_tokens=True)
                for idx, text in zip(batch_idxs, outs):
                    records.append({
                        "id": int(df.at[idx, "id"]),
                        "author": df.at[idx, "author"],
                        "output": text
                    })
                clean_batches += 1
                pbar.update(len(batch_idxs))

            except RuntimeError as e:
                if "out of memory" in str(e).lower() and len(batch_idxs) > 1:
                    torch.cuda.empty_cache(); gc.collect()
                    current_budget = max(current_budget // 2, tok_sum)
                    clean_batches = 0
                    print(f"⚠️ OOM -> new budget {current_budget}")
                    mid = len(batch_idxs) // 2
                    for half in (batch_idxs[:mid], batch_idxs[mid:]):
                        half_enc = {k: batch[k][:len(half)] for k in batch}
                        half_tok = int(half_enc["attention_mask"].sum())
                        batch_queue.put((half_enc, half, half_tok, _))
                else:
                    print(f"Error: {e}"); traceback.print_exc()
                    clean_batches = 0

            if clean_batches >= budget_update_freq and pct < memory_usage_threshold:
                current_budget += budget_update_quota
                print(f"⚡ budget -> {current_budget}")
                clean_batches = 0

            print(f"complete batch {n} processing...")
            n += 1

        pbar.close()

    except KeyboardInterrupt:
        print("⚠️ Interrupted — saving partial results...")

    finally:
        stop_event.set()
        prod_thread.join()
        try:
            df_out = original.merge(pd.DataFrame(records), on="id", how="left")
            with open(output_log_path, "w", encoding="utf-8") as f:
                json.dump(df_out.to_dict(orient="records"), f, ensure_ascii=False, indent=2)
            print(f"✅ Saved {len(records)} records")
        except Exception as ex:
            print(f"❌ Save failed: {ex}")
        finally:
            pynvml.nvmlShutdown()

    return records


In [22]:
import torch
torch.cuda.empty_cache()

import gc
gc.collect()

30

In [23]:
!nvidia-smi

Thu Jul 24 14:56:01 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla P100-PCIE-16GB           Off |   00000000:00:04.0 Off |                    0 |
| N/A   38C    P0             25W /  250W |       3MiB /  16384MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [24]:
# !watch -n 0.5 nvidia-smi
torch.cuda.is_available()  # Should be True
torch.cuda.current_device()
torch.cuda.get_device_name()

'Tesla P100-PCIE-16GB'

In [25]:
## gpu usage rate  logging test

# import torch
# import time

# torch.cuda.empty_cache()
# a = torch.randn(10000, 10000, device="cuda")

# # Warm-up
# for _ in range(5):
#     a = torch.matmul(a, a)

# # Now measure and keep GPU busy for a while
# start = time.time()

# for _ in range(100):
#     a = torch.matmul(a, a)
#     torch.cuda.synchronize()  # make it block until GPU is done

# end = time.time()
# print(f"Time taken: {end - start:.2f} seconds")



# import torch
# import pynvml
# import time

# # Start NVML
# pynvml.nvmlInit()
# handle = pynvml.nvmlDeviceGetHandleByIndex(0)

# # Heavy GPU computation
# a = torch.randn(8192, 8192, device="cuda")
# b = torch.randn(8192, 8192, device="cuda")

# # Run and monitor in real-time
# for _ in range(10):
#     c = torch.matmul(a, b)  # triggers real GPU usage
#     util = pynvml.nvmlDeviceGetUtilizationRates(handle)
#     print(f"GPU Usage: {util.gpu}% | Memory Usage: {util.memory}%")
#     time.sleep(0.5)

# pynvml.nvmlShutdown()

In [26]:
# ---------------- Cell 4: run the pipeline ----------------

# if __name__ == '__main__':
# adjust these paths as needed
test_file = "./data/PublicDataFinalPhaseTask1.xlsx"
log_file = f"models/fine_tuned_model/results/{safe_model_name(MODEL_NAME)}/generation_log_test.json"
output_base_dir = "analysis/fine_tuned_model"

os.makedirs(os.path.dirname(log_file), exist_ok=True)
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"


tokenizer, model = load_model(MODEL_NAME)

# base_model_path = "models/base_models/facebook/arabart"
# lora_path       = "models/fine_tuned_model/model_weights/lora_moussa_arabart"

# tokenizer, model = load_lora_model(base_model_path, lora_path)


Loading model from models/fine_tuned_model/model_weights/UBC-NLP_AraT5v2-base-1024


In [ ]:
# generate_predictions(tokenizer, model, test_file, log_file, batch_size=128)
out_df = generate_predictions_by_budget(tokenizer, model, test_file, log_file, token_budget=15_000)

▶ model.max_position_embeddings = 100000000 , safe_input = 3400, safe_generation = 4000
loading and tokenization start ...
loading and tokenization done ... loading the model to gpu...


  0%|          | 0/8413 [00:00<?, ?it/s]

retrieve batch 0 ...
start batch 0 processing...
 min gen = 201 , max_new=257 , max_gen = 257
[Gen] 87 samples | Time: 68.5s | VRAM: 13465MB (78%)


  1%|          | 87/8413 [01:10<1:52:17,  1.24it/s]

complete batch 0 processing...
retrieve batch 0 ...
start batch 0 processing...
 min gen = 240 , max_new=308 , max_gen = 308
[Gen] 59 samples | Time: 65.6s | VRAM: 13465MB (78%)


...

complete batch 0 processing...
retrieve batch 0 ...
start batch 0 processing...
 min gen = 320 , max_new=410 , max_gen = 410
[Gen] 44 samples | Time: 71.8s | VRAM: 13467MB (78%)


  6%|▌         | 511/8413 [11:51<3:33:46,  1.62s/it]

⚡ budget -> 17000
complete batch 0 processing...
retrieve batch 0 ...
start batch 0 processing...
 min gen = 325 , max_new=416 , max_gen = 416
[Gen] 43 samples | Time: 68.6s | VRAM: 13467MB (78%)


  7%|▋         | 554/8413 [13:01<3:32:47,  1.62s/it]

complete batch 0 processing...
retrieve batch 0 ...
start batch 0 processing...
 min gen = 328 , max_new=420 , max_gen = 420


In [ ]:
def print_first_two_results(records):
    print("\n📄 First Two Output Records:\n" + "-"*30)
    for i, rec in enumerate(records[:2]):
        print(f"\n📝 Record #{i+1}")
        for key, value in rec.items():
            print(f"{key}: {value}")


# print_first_two_results(out_df)

In [ ]:
# records, df = generate_predictions(tokenizer, model, validation_file, log_file)
# If saving separately is needed
# save_predictions_to_file(records, df, log_file)

run_dir = create_run_dir(output_base_dir, MODEL_NAME)
os.system(f"cp {log_file} {run_dir}/")
print(f"\n✅ Pipeline done. Logs in: {run_dir} \n LOG FILE: {log_file}\n")

In [ ]:


# Function to regenerate missing samples one-by-one with stale‐assert flush
def regenerate_missing(
    test_file,
    log_file,
    tokenizer,
    model,
    device,
    max_input_length=3350,
    max_generation_length=3500
):
    import json, time, torch, pandas as pd
    from tqdm import tqdm

    max_pos = model.config.max_position_embeddings
    # min(max_input_length, max_pos)
    gen_max = min(max_generation_length, max_pos)
    print(f"▶ model.max_position_embeddings = {max_pos}")

    df_test = pd.read_excel(test_file)[['id','author','text_in_msa']].astype({'id':int})
    try:
        existing = pd.DataFrame(json.load(open(log_file,'r',encoding='utf-8')))
    except FileNotFoundError:
        existing = pd.DataFrame(columns=['id','author','output'])
    existing['id'] = existing['id'].astype(int)

    merged = df_test.merge(existing, on=['id','author'], how='left')
    missing = merged[ merged['output'].isna() | (merged['output'].str.strip()=='') ]
    print(f"🔍 {len(missing)} missing samples to regenerate")

    new_records = []
    for _, row in tqdm(missing.iterrows(), total=len(missing), desc='Regenerating'):
        # 1) Clear any *stale* device‐side assert
        try:
            torch.cuda.synchronize()
        except Exception:
            pass
        torch.cuda.empty_cache()

        text = f"{row['author']} [SEP] {row['text_in_msa']}"
        enc = tokenizer(
            text,
            return_tensors='pt',
            truncation=True,
            max_length=min(max_input_length, max_pos),
            padding=False
        )

        try:
            # 2) Move to GPU and generate
            enc = {k: v.to(device) for k,v in enc.items()}
            input_len = enc['input_ids'].size(1)
            # gen_max   = min(max_generation_length, max_pos - input_len)
            gen_max = min(max_generation_length, max_pos)
            if gen_max < 1:
                raise RuntimeError(f"Input length {input_len} ≥ max_pos {max_pos}")

            with torch.inference_mode():
                torch.cuda.synchronize()
                t0 = time.time()
                out_ids = model.generate(
                    input_ids=enc['input_ids'],
                    attention_mask=enc.get('attention_mask'),
                    max_new_tokens=gen_max,
                    use_cache=True,
                    num_beams=1,
                    pad_token_id=tokenizer.pad_token_id
                )
                torch.cuda.synchronize()
                dt = time.time() - t0

            decoded = tokenizer.decode(out_ids[0], skip_special_tokens=True)
            print(f"✓ ID {row['id']} ({input_len}+{gen_max})→{out_ids.size(1)} in {dt:.1f}s")

        except Exception as e:
            print(f"❌ ID {row['id']} failed: {e}")
            decoded = ""

        new_records.append({'id':int(row['id']), 'author':row['author'], 'output':decoded})

    # Merge back & save
    all_df = pd.concat([existing, pd.DataFrame(new_records)], ignore_index=True)
    out_df = df_test.merge(all_df[['id','author','output']], on=['id','author'], how='left')

    with open(log_file,'w',encoding='utf-8') as f:
        json.dump(out_df.to_dict(orient='records'), f, ensure_ascii=False, indent=2)

    print(f"✅ Regeneration complete. {len(out_df)} rows in log.")
    return out_df

# Run it
out_df = regenerate_missing(
    test_file=test_file,
    log_file=log_file,
    tokenizer=tokenizer,
    model=model,
    device=next(model.parameters()).device,
    max_input_length=3500,
    max_generation_length=4000
)


In [ ]:
!nvidia-smi

In [ ]:
from IPython.display import FileLink

# Display a link to the copied log file in the run_dir
log_file_in_run_dir = os.path.join(run_dir, os.path.basename(log_file))
FileLink(log_file_in_run_dir)

In [ ]:
# import torch
# import gc

# def flush_gpu_memory():
#     import torch, gc
#     gc.collect()
#     torch.cuda.empty_cache()

#     # Kill CUDA tensors
#     for obj in gc.get_objects():
#         try:
#             if torch.is_tensor(obj) or (hasattr(obj, 'data') and torch.is_tensor(obj.data)):
#                 if obj.is_cuda:
#                     del obj
#         except Exception:
#             pass

#     gc.collect()
#     torch.cuda.empty_cache()

#     # Clear CUDA context if possible (dangerous but useful in some cases)
#     try:
#         torch.cuda.ipc_collect()
#     except:
#         pass


# flush_gpu_memory()


# ######

# # Step 1: Delete large objects
# # del model, tokenizer

# # # Step 2: If model is still in GPU, move to CPU first
# # model.to("cpu")  
# # # then 
# # del model

# # Step 3: Run GC and empty cache
# import gc, torch
# gc.collect()
# torch.cuda.empty_cache()

# # Step 4: Confirm
# print(f"Allocated: {torch.cuda.memory_allocated() / 1e6:.1f} MB")
# print(f"Cached:    {torch.cuda.memory_reserved() / 1e6:.1f} MB")


# # !nvidia-smi --gpu-reset -i 0

In [ ]:
# # Function to regenerate missing samples one-by-one with the same budgeted generation settings
# def regenerate_missing(test_file, log_file, tokenizer, model, device,
#                        max_input_length=3350, max_generation_length=3500,
#                        token_budget=30000):
#     import json, time, torch, pandas as pd
#     from tqdm import tqdm

#     # Load test file and existing log
#     df_test = pd.read_excel(test_file)
#     df_test = df_test[["id", "author", "text_in_msa"]]
#     df_test["id"] = df_test["id"].astype(int)
#     try:
#         with open(log_file, "r", encoding="utf-8") as f:
#             existing = pd.DataFrame(json.load(f))
#     except FileNotFoundError:
#         existing = pd.DataFrame(columns=["id", "author", "output"]);

#     existing["id"] = existing["id"].astype(int)
#     # Identify missing IDs (no output or empty)
#     merged = df_test.merge(existing, on=["id", "author"], how="left")
#     missing = merged[merged["output"].isna() | (merged["output"].str.strip() == "")]
#     missing_ids = missing["id"].tolist()
#     print(f"🔍 {len(missing_ids)} missing samples to regenerate")

#     # Regenerate one-by-one
#     new_records = []
#     for _, row in tqdm(missing.iterrows(), total=len(missing), desc="Regenerating missing"):
#         input_text = row["author"] + " [SEP] " + row["text_in_msa"]
#         enc = tokenizer(
#             input_text,
#             truncation=True,
#             max_length=max_input_length,
#             return_tensors="pt",
#             padding=True
#         ).to(device)
#         try:
#             with torch.no_grad():
#                 torch.cuda.synchronize()
#                 t0 = time.time()
#                 out_ids = model.generate(
#                     input_ids=enc["input_ids"],
#                     attention_mask=enc["attention_mask"],
#                     max_new_tokens=max_generation_length,
#                     use_cache=True,
#                     num_beams=1
#                 )
#                 torch.cuda.synchronize()
#                 dt = time.time() - t0
#             decoded = tokenizer.decode(out_ids[0], skip_special_tokens=True)
#             print(f"✓ ID {row['id']} generated in {dt:.1f}s")
#         except Exception as e:
#             print(f"❌ ID {row['id']} failed: {e}")
#             decoded = ""
#         new_records.append({"id": int(row["id"]), "author": row["author"], "output": decoded})

#     # Combine and reorder
#     all_df = pd.concat([existing, pd.DataFrame(new_records)], ignore_index=True)
#     all_df = df_test.merge(all_df, on=["id", "author"], how="left")
#     # Save back to log_file in original test_file order
#     records = all_df[["id", "author", "output"]].to_dict(orient="records")
#     with open(log_file, "w", encoding="utf-8") as f:
#         json.dump(records, f, ensure_ascii=False, indent=2)
#     print(f"✅ Regeneration complete. Log updated with {len(records)} records.")

# # 2) Regenerate missing samples
# regenerate_missing(
#     test_file=test_file,
#     log_file=log_file,
#     tokenizer=tokenizer,
#     model=model,
#     device=next(model.parameters()).device,
#     max_input_length=3350,
#     max_generation_length=3500,
#     token_budget=30_000
# )

# # 3) Archive log file
# os.system(f"cp {log_file} {run_dir}/")
# print(f" ✅ Pipeline done. Logs in: {run_dir} LOG FILE: {log_file}")


In [ ]:
# # For Colab or IPython environments
# import os
# os._exit(00)